In [156]:
import pandas as pd
import numpy as np
df=pd.read_csv("combined.csv")

def convertToFloat(x):
    x=x.strip()
    if x=="-" or x=="NR" or x=='' or x=="." or x=="" or x==' ':
        return 0.01
    else:
        
        return float(x)

In [157]:
df["rain"]=df["Rain"].apply(convertToFloat)
import datetime
df["month"]=df["Date"].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").month)
df["year"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").year)
df["day"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").day)

In [158]:
def isRain(x):
    if x!=0.01:
        return 1
    else:
        return 0

In [159]:
df["is_rain"]=df["rain"].apply(isRain)

In [160]:
dummydf=pd.get_dummies(df[["District","rain","month","day","year","is_rain"]])

In [146]:
Al_df=df[df.District=="Alangayam"][["rain","month","day","year","is_rain"]]

In [161]:
y=dummydf.pop("rain")
x=dummydf

In [148]:
y=Al_df.pop("rain")
x=Al_df
X_train, X_test, y_train, y_test = train_test_split(x , y, test_size=0.33, random_state=42) 

In [162]:
 from sklearn.model_selection import train_test_split
 from sklearn.preprocessing import StandardScaler

In [163]:
X_train, X_test, y_train, y_test = train_test_split(x , y, test_size=0.33, random_state=42)

In [131]:
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
scaler2 = StandardScaler()

X_test=scaler2.fit_transform(X_test)

In [164]:
X_train

,month,day,year,is_rain,District_ACS Mill,District_Alangayam,District_Ambur,District_Arakkonam,District_Arcot,District_Gudiyatham,...,District_Melalathur,District_Natrampalli,District_Ponnai,District_Sholingur,District_TCS Mill,District_Tirupattur,District_VCS Mill,District_Vaniyambadi,District_Vellore,District_Wallajah
29085,9,12,2016,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
23469,6,2,2015,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
26637,2,21,2016,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1351,5,3,2010,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1521,5,19,2010,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,8,21,2012,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
44732,10,6,2019,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
38158,10,6,2018,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
860,3,20,2010,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
import xgboost as xg

ModuleNotFoundError: No module named 'xgboost'

In [ ]:
!pip3 install xgboost

In [165]:
from sklearn.linear_model import LinearRegression

In [166]:
reg = LinearRegression().fit(X_train, y_train)

In [167]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,reg.predict(X_test))

2.1710671472047034

In [128]:
max(y_test[0])

0.0

In [104]:
reg.predict(X_train)

array([ 0.21894508, 12.46740162, -0.14647121, ..., 12.94425225,
        0.38613749,  0.3628996 ])

In [31]:
import xgboost as xg

In [118]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.]])

In [184]:
xgb_r = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 50, seed = 123,max_depth=20)
   
# Fitting the model
xgb_r.fit(X_train, y_train)

[16:34:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=20,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=12, num_parallel_tree=1,
             objective='reg:linear', random_state=123, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=123, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [185]:
pred = xgb_r.predict(X_test)


C:\Users\om purohit\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [35]:
def mean_absolute_percentage_error(y_true, y_pred): 
    # y_true, y_pred = check_arrays(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [41]:
mean_absolute_percentage_error( y_train,pred)

inf

In [140]:
mean_absolute_error(y_test,pred)

1.63591009423641

In [173]:
pred_test=xgb_r.predict(X_test)

C:\Users\om purohit\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [174]:
mean_absolute_error(y_test,pred_test)

1.8033440438697161

In [73]:
d={"orig":y_test,"pred_test":pred_test}
preddf=pd.DataFrame(d)

In [74]:
preddf[preddf["orig"]!=0.0]

,orig,pred_test
8140,56.00,25.875324
2059,4.20,18.684149
16460,11.08,8.325056
34059,1.20,27.906496
33698,3.60,4.866726
...,...,...
33197,5.00,11.482909
16177,8.40,5.265359
6594,25.40,17.836197
32460,26.80,26.967621


In [179]:
from sklearn.svm import LinearSVR
eps = 5
svr = LinearSVR(epsilon=eps, C=0.01, fit_intercept=True)

In [180]:
svr.fit(X_train,y_train)

C:\Users\om purohit\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVR(C=0.01, epsilon=5)

In [183]:
mean_absolute_error(y_test,svr.predict(X_test))

5.816683235951595

In [186]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,pred)

50.42033054947053